In [1]:
import torch
import math
import torch.nn as nn
import torch.optim as optim
import torch.utils
import PIL
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from torchvision import datasets
import d2l
from d2l import torch as d2l
import datetime
import torch.nn.functional as F

In [2]:
#Downloading CIFAR-10
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) #下载太慢请开代理

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 引入normalize的数据初始化
tensor_cifar10_normalize_train = datasets.CIFAR10(data_path, train=True, download=False,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                     (0.2470, 0.2435, 0.2616))
                            ]))

tensor_cifar10_normalize_val = datasets.CIFAR10(data_path, train=True, download=False,
                            transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                     (0.2470, 0.2435, 0.2616))
                            ]))

In [4]:
# Build the dataset and DataLoader

label_map = {0: 0, 2: 1} # 占位符
class_names = ['airplane', 'bird']
# 训练集
cifar2 = [(img, label_map[label])
    for img, label in tensor_cifar10_normalize_train
        if label in [0, 2]]
# 验证集
#cifar2_val = [(img, label_map[label])
#   for img, label in tensor_cifar10_normalize_val
#      if label in [0, 2]]

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

In [5]:
# train_loop
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        
        for imgs, labels in train_loader:
            imgs = imgs.cuda()
            labels = labels.cuda()

            batch_size = imgs.shape[0]
            outputs = model(imgs.view(batch_size, -1))
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [6]:
model = nn.Sequential(
        nn.Linear(3072, 512),
        nn.Tanh(),
        nn.Linear(512, 2),
        nn.LogSoftmax(dim=1))

model.cuda()

optimizer = optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.NLLLoss()

In [7]:
training_loop(
            n_epochs = 100,
            optimizer = optimizer,
            model = model,
            loss_fn = loss_fn,
            train_loader = train_loader,
            )

2021-10-15 03:09:18.204090 Epoch 1, Training loss 0.5083087659945154
2021-10-15 03:09:21.482491 Epoch 10, Training loss 0.3811726199973161
2021-10-15 03:09:25.087902 Epoch 20, Training loss 0.30093338819825727
2021-10-15 03:09:28.653624 Epoch 30, Training loss 0.2129609514099018
2021-10-15 03:09:32.220413 Epoch 40, Training loss 0.1421883511505309
2021-10-15 03:09:35.879727 Epoch 50, Training loss 0.09099124941476591
2021-10-15 03:09:39.516388 Epoch 60, Training loss 0.05940437073682903
2021-10-15 03:09:43.147935 Epoch 70, Training loss 0.04027432285154321
2021-10-15 03:09:46.744683 Epoch 80, Training loss 0.028183614793619152
2021-10-15 03:09:50.374837 Epoch 90, Training loss 0.02366593038176845
2021-10-15 03:09:53.946078 Epoch 100, Training loss 0.017173104014864583
